# eBay Sales Analysis

In [8]:
import requests
#BeautifulSoup es una calse dentro de la libreria bs4
from bs4 import BeautifulSoup
import pandas as pd
# from pprint import pprint
# from lxml import html
# from lxml.html import fromstring
# import urllib.request
from urllib.request import urlopen
# import random
import re
# import scrapy
import csv
import datetime

1. We analize the html code to searh for patterns to extract the info that qe need

In [9]:
url = "https://www.ebay.com/sch/i.html?_dcat=15687&_fsrp=1&rt=nc&_from=R40&_nkw=vintage+t+shirt&_sacat=0&_ipg=240&LH_Sold=1&Theme=Band"

In [10]:
html = requests.get(url)
html

<Response [200]>

In [11]:
items = BeautifulSoup(html.content, "html.parser")

In [12]:
entries = items.find_all("div", {"class":"s-item__info clearfix"})
title = entries[1].find_all("div", {"class":"s-item__title"})[0].getText()
sold =  entries[1].find_all("span", {"class":"POSITIVE"})[0].getText()
price = entries[1].find_all("span", {"class":"s-item__price"})[0].getText()
shipping_cost =  entries[1].find_all("span", {"class":"s-item__shipping s-item__logisticsCost"})[0].getText()
shipped_from =  entries[1].find_all("span", {"class":"s-item__location s-item__itemLocation"})[0].getText()
print(title)
print(sold)
print(price)
print(shipping_cost)
print(shipped_from)

New ListingVintage 1992 Pearl Jam Choices T Shirt XL Fruit of the Loom 9 out of 10 Original
Sold  Feb 4, 2024
$233.54
+$31.42 shipping
from United States


2. We use a predefined link, that we will modiffy for each page of aur tageted search

In [13]:
# with this variable we can change the page in with we srpape
n = 1

In [14]:
# formated link --> with page variable
url = f"https://www.ebay.com/sch/i.html?_dcat=15687&_fsrp=1&rt=nc&_from=R40&_nkw=vintage+t+shirt&_sacat=0&_ipg=240&LH_Sold=1&Theme=Band&_pgn={n}"

In [15]:
# We extract the html code
html = requests.get(url)
html

<Response [200]>

In [16]:
# parse it with beautifull soup
items = BeautifulSoup(html.content, "html.parser")

In [19]:
# We apply a function that extracts and sets to a df all selected values in the page 
sold = page_table(items, "Music")

In [20]:
#DataFrame generated

sold

#I applied this method for 1 to 26 pages and then concatenated
#all data frames generated into one and exported it as .csv

,description,sold_date,price,shipping,location,collection
0,LOT OF 12 VINTAGE STYLE CLASSIC RAP HIP-HOP UR...,"Sold Feb 4, 2024",$85.00,+$52.43 shipping,from United States,Music
1,Metallica Large Vintage T Shirt Single Stitch ...,"Sold Feb 4, 2024",$34.81,+$13.24 shipping estimate,from United Kingdom,Music
2,Outkast Band Graphic T Shirt Target Vintage St...,"Sold Feb 4, 2024",$5.00,Shipping not specified,from United States,Music
3,Vintage Columbine High School Columbine Open S...,"Sold Feb 4, 2024",$65.00,+$26.72 shipping,from United States,Music
4,Vintage Hanes Shirt Adult Small 80's 1983 Shor...,"Sold Feb 4, 2024",$29.99,+$19.00 shipping,from United States,Music
...,...,...,...,...,...,...
235,Vintage 90s Sting /the Police Band/ Black T-Sh...,"Sold Feb 3, 2024",$15.00,+$32.00 shipping,from United States,Music
236,U2 Vertigo 2005 Tour 3/4 Sleeve Raglan Band T-...,"Sold Feb 3, 2024",$30.00,+$25.69 shipping,from United States,Music
237,BLINK 182 MENS RARE BAND T-SHIRT NEW SIZE L Vi...,"Sold Feb 3, 2024",$12.99,+$19.33 shipping,from United States,Music
238,Vintage Bon Jovi Shirt Bounce Tour XL Thunder ...,"Sold Feb 3, 2024",$24.00,+$28.22 shipping,from United States,Music


In [18]:
def page_table(soup, collection):
    
    """Returns a DataFrame for a given soup"""
    
    #1. Fist we target the box elements where all the info of a product is contained:
    elemnt_box = soup.find_all("div", {"class":"s-item__info clearfix"})
    
    elemnt_box = elemnt_box[1:]
    #2. We iterate trouth the elements to extract the info that we need: Comprehension lists
    
    #2.1 Descriptions list:
    description = [ i.find("div", {"class":"s-item__title"}).getText().strip() for i in elemnt_box]
    
    #2.2 Date:
    date = [i.find("span", {"class":"POSITIVE"}).getText().strip() for i in elemnt_box]
    
    
    #2.2 Prices list:
    price = [ i.find("span", {"class":"s-item__price"}).getText().strip() for i in elemnt_box]
    
    #2.3 shipping:
    shipping = [ i.find("span", {"class":"s-item__shipping s-item__logisticsCost"}).getText().strip() for i in elemnt_box]
    
    # seller 
    shipped_from = [ i.find("span", {"class":"s-item__location s-item__itemLocation"}).getText().strip() for i in elemnt_box]
    
    
    
    #3. Create a disctionary with the tree lists:
    page_dict = {}
    
    page_dict["description"] = description
    page_dict["sold_date"] = date
    page_dict["price"] = price
    page_dict["shipping"] = shipping
    page_dict["location"] = shipped_from
    page_dict["collection"] = [collection for i in range(len(description))]

    
    df = pd.DataFrame(page_dict)
    
    return df

In [228]:
def export_to_csv(data_frame, file_name):
    """
    Export a DataFrame to a CSV file.

    Parameters:
        - data_frame: pandas DataFrame
        - file_name: str, name of the CSV file (without the extension)
    """
    #print date stamp - datetime.datetime.now()
    
    t_stamp = str(datetime.datetime.now()).split(".")[0].replace(" ","_")
    
    file_path = f"{file_name}_{t_stamp}.csv"
    
    data_frame.to_csv(file_path, index=False)
    print(f"DataFrame successfully exported to {file_path}")